<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
    TIME SERIES DATA PROCESSING
    <br>
    OUTAGE FACTORS
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    Main Formatting Notebook
</div>
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Each part of the following script was used to proccess the raw data for the Outage Factors Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
import re
import csv
import os
import requests
from urllib.parse import urlparse, parse_qs
import datetime
from bs4 import BeautifulSoup
import http.client
from multiprocessing import Pool
import shutil
import numpy as np
from entsoe import EntsoePandasClient
from datetime import datetime
import pytz
from datetime import datetime, timedelta

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: jusitfy; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    3. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Indicate the year of all data is referring to in the variable data_year.
<br>
The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
</div>

In [3]:
# Year to which data refers to:
data_year = 2016

# Universal standad time:
universal_standard_time = 'UTC'

# Western European Time:
western_european_time = 'WET_WEST'

# Central European Time:
central_european_time = 'CET_CEST'

# Eastern European Time:
eastern_european_time = 'EET_EST'

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
4. Outage Factors Directories Definition
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Creating the folders that are going to content all the data realted to the Outage Factors Level time series.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Futher the downloaded raw data is going to be used to get the Outage Factors time series. 
</div>

In [4]:
# Additional string to be appended
additional_path = "/RawData/OutageFactors/"
additional_path_1 = "/RawData/OutageFactors/Raw_Data_Sources/"
additional_path_2 = "/Database/AvailabilityFactors/"
additional_path_3 = "/Database/PowerPlants/"

# Construct the standard_time_data_folder_path variable
reference_data_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_folder_path variable
outage_factors_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_Raw_Data_folder_path variable
outage_factors_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_1 + str(data_year)

# Construct the Availability_Factors_Data_folder_path variable
availability_factors_data_folder_path = dispaSET_unleash_folder_path + additional_path_2

# Construct the Power_Plants_Data_folder_path variable
power_plants_data_folder_path = dispaSET_unleash_folder_path + additional_path_3


print("reference_data_folder_path:", reference_data_folder_path)
print("outage_factors_folder_path:", outage_factors_folder_path)
print("outage_factors_raw_data_folder_path:", outage_factors_raw_data_folder_path)
print("availability_factors_data_folder_path:", availability_factors_data_folder_path)
print("power_plants_data_folder_path:", power_plants_data_folder_path)

reference_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/
outage_factors_folder_path: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/
outage_factors_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Raw_Data_Sources/2016
availability_factors_data_folder_path: /home/ray/Dispa-SET_Unleash/Database/AvailabilityFactors/
power_plants_data_folder_path: /home/ray/Dispa-SET_Unleash/Database/PowerPlants/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    4.1. Back Up Directory
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Saving the original files into a Back up folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since in the next steps of the processing data new subfolders and files are going to be created, the original ones are saved in a back up foldet to return them as its default content ones the process will be finished.
</div>

In [5]:
additional_path_5 = '/RawData/OutageFactors_backup/'

# Construct the backup_folder_path variable
backup_folder_path = dispaSET_unleash_folder_path + additional_path_5

print("backup_folder_path:", backup_folder_path)

# Create a backup of the directory
if os.path.exists(backup_folder_path):
    shutil.rmtree(backup_folder_path)  # Remove any existing backup if necessary
shutil.copytree(outage_factors_folder_path, backup_folder_path)

print(f"Backup created at {backup_folder_path}")

backup_folder_path: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors_backup/
Backup created at /home/ray/Dispa-SET_Unleash/RawData/OutageFactors_backup/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
4. European Standard Time
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
All the time series are going to be downloaded under the UTC <em>(The World’s Time Standard)</em>.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
This is going to be done for each country as well.
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
All this features are going to be saved in a csv file called Reference_Data.csv where all additional characters to the download process are going to be written.
</div>

In [6]:
# Define lists of countries and standard times
countries = [
    "Austria", "Belgium", "Bulgaria", "Switzerland", "Cyprus", "Czech Republic",
    "Germany", "Denmark", "Estonia", "Greece", "Spain", "Finland", "France",
    "Croatia", "Hungary", "Ireland", "Italy", "Lithuania", "Luxembourg", "Latvia",
    "Malta", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Sweden",
    "Slovenia", "Slovakia", "United Kingdom"
]

dispaSET_codes = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "EL", "ES", "FI", "FR", "HR", "HU", 
                  "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"
]


# Create DataFrame
df = pd.DataFrame({'Country': countries, 'Dispa-SET_Code': dispaSET_codes})

reference_data_file_name = 'Reference_Data.csv'

# Construct the full file path
reference_data_file_path = os.path.join(reference_data_folder_path, reference_data_file_name)

# Create the CSV file with the specified name
with open(reference_data_file_path, 'w') as f:
    # Optional: Write a header if needed
    # f.write("header1,header2,header3\n")

# Save DataFrame to the CSV file
    df.to_csv(reference_data_file_path, index=False)

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [7]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"universal_standard_time:                                   {universal_standard_time}")
print (f"western_european_time:                                     {western_european_time}")
print (f"central_european_time:                                     {central_european_time}")
print (f"eastern_european_time:                                     {eastern_european_time}")
print (f"reference_data_folder_path:                                {reference_data_folder_path}")
print (f"reference_data_file_name:                                  {reference_data_file_name}")
print (f"reference_data_file_path:                                  {reference_data_file_path}")
print (f"outage_factors_folder_path:                                {outage_factors_folder_path}")
print(f"outage_factors_raw_data_folder_path:                        {outage_factors_raw_data_folder_path}")
print(f"power_plants_data_folder_path:                              {power_plants_data_folder_path}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2016
universal_standard_time:                                   UTC
western_european_time:                                     WET_WEST
central_european_time:                                     CET_CEST
eastern_european_time:                                     EET_EST
reference_data_folder_path:                                /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/
reference_data_file_name:                                  Reference_Data.csv
reference_data_file_path:                                  /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Reference_Data.csv
outage_factors_folder_path:                                /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/
outage_factors_raw_data_folder_path:                        /home/ray/Dispa-SET_

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
5. Main Data Source
</div>

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Sellecting the main source to get the raw data of the time series.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
All the data to be processed was downloaded from one main source:
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
<em><strong>ENTSOE Transparency Platform:</strong></em>
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Which the main url link is the following
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
https://transparency.entsoe.eu/dashboard/show
</div>
<br>
<div style="text-align: jusitfy; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.1. Raw Data Sources Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Entering all the files paths where the raw data is content.
     <br>
        All the csv files with the outages raw data are storaged a folder with the name of the corresponding year in a local folder which path is:
    <div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
/Local/Dispa_Set/Path/Dispa-SET_Unleash/RawData/OutageFactors/Raw_Data_Sources/
        </div>
The path of this csv files are going to be added to the references data csv file for manipulation purposes. 
</div>

In [8]:
# Define file paths
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Reference_Data.csv"
#outage_factors_raw_data_folder_path = "/home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Raw_Data_Sources"

# Read the reference data CSV file
reference_data = pd.read_csv(reference_data_file_path)

# Function to get file path from Dispa-SET code
def get_file_path(code):
    file_name = code + ".csv"
    file_path = os.path.join(outage_factors_raw_data_folder_path, file_name)
    return file_path if os.path.exists(file_path) else None

# Add a new column 'Raw_Data_File_Path' and fill it
reference_data['Raw_Data_File_Path'] = reference_data['Dispa-SET_Code'].apply(get_file_path)

# Save the updated dataframe back to the CSV file
reference_data.to_csv(reference_data_file_path, index=False)

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.2. Unnecessary Raw Data 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Eliminating those rows and columns of the raw data files that content unrerquired and repeated data. 
</div>

In [9]:
# Define the path to the reference data CSV file
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Reference_Data.csv"

# Read the reference data CSV file into a DataFrame
df_reference = pd.read_csv(reference_data_file_path)

# Define the column to process which contains file paths to other CSV files
column_to_process = 'Raw_Data_File_Path'

# Iterate over each file path in the specified column
for index, file_path in df_reference[column_to_process].dropna().items():
    # Check if the file exists
    if os.path.exists(file_path):
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Remove rows where the docstatus column has the value 'Cancelled'
        df = df[df['docstatus'] != 'Cancelled']
        
        # Construct the new file path with '_1' suffix
        base, ext = os.path.splitext(file_path)
        new_file_path = f"{base}_1{ext}"
        
        # Save the modified CSV file to the new file path
        df.to_csv(new_file_path, index=False)
        print(f"Processed and saved: {new_file_path}")
        
        # Update the path in the reference DataFrame
        df_reference.at[index, column_to_process] = new_file_path
    else:
        print(f"File not found: {file_path}")

# Save the updated reference DataFrame back to the reference data file
df_reference.to_csv(reference_data_file_path, index=False)
print("Task completed.")

Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Raw_Data_Sources/2016/AT_1.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Raw_Data_Sources/2016/BE_1.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Raw_Data_Sources/2016/BG_1.csv
Processed and saved: /home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Raw_Data_Sources/2016/CH_1.csv


KeyError: 'docstatus'

In [ ]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the column containing file paths to process
column_to_process = 'Raw_Data_File_Path'

# Define the columns to keep in the corresponding CSV files
columns_to_keep = ['avail_qty', 'end', 'nominal_power', 'production_resource_name', 'start']

# Iterate over each file path in the specified column
for file_path in df_reference[column_to_process].dropna():
    # Check if the file exists
    if os.path.exists(file_path):
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if all specified columns exist in the DataFrame
        if all(column in df.columns for column in columns_to_keep):
            # Keep only the specified columns
            df = df[columns_to_keep]
            
            # Save the modified CSV file back to its original location
            df.to_csv(file_path, index=False)
            print(f"Processed and saved: {file_path}")
        else:
            print(f"Columns not found in file {file_path}. Skipping.")
    else:
        print(f"File not found: {file_path}")

print("Task completed.")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.3. UTC Time Convertion
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Changing the dates to the UTC time.
</div>

In [ ]:
# Define the reference data CSV file path
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Reference_Data.csv"

# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the column to process
column_to_process = 'Raw_Data_File_Path'

# Function to convert a timestamp to UTC
def convert_to_utc(timestamp_str):
    local_time = datetime.fromisoformat(timestamp_str)
    utc_time = local_time.astimezone(pytz.utc)
    return utc_time.strftime('%Y-%m-%d %H:%M:%S%z')

# Function to add a colon at the third position from the end
def add_colon_third_from_end(timestamp_str):
    return timestamp_str[:-2] + ':' + timestamp_str[-2:]

# Iterate over each file path in the specified column
for file_path in df_reference[column_to_process].dropna():
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Convert the 'start' and 'end' columns to UTC
        df['start'] = df['start'].apply(convert_to_utc)
        df['end'] = df['end'].apply(convert_to_utc)
        
        # Add a colon at the third position from the end
        df['start'] = df['start'].apply(add_colon_third_from_end)
        df['end'] = df['end'].apply(add_colon_third_from_end)
        
        # Save the modified CSV file back to its original location
        df.to_csv(file_path, index=False)
        print(f"Processed and saved: {file_path}")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

print("Task completed.")

<div style="text-align: Justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.4. Yearly Raw Data
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Sellecting the raw data inside the analized period of time (defined by the data_year variable).
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Some availabilities are given in dates outside the analyzed time period. e.g. started in a previous year or ended in a year after.
Just those periods that are inside the analyzed year are going to be kept. 
</div>

In [ ]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the column containing the file paths
column_to_process = 'Raw_Data_File_Path'

# Function to update timestamps based on data year
def update_timestamps(timestamp_str):
    year_part = str(data_year)
    start_of_year = f"{year_part}-01-01 00:00:00+00:00"
    end_of_year = f"{year_part}-12-31 23:59:00+00:00"
    
    # Check if the timestamp is outside the data year range
    if not (start_of_year <= timestamp_str <= end_of_year):
        return start_of_year if timestamp_str < start_of_year else end_of_year
    
    # If the timestamp is within the data year range, keep it unchanged
    return timestamp_str

# Iterate over each file path in the specified column
for file_path in df_reference[column_to_process].dropna():
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"Error: File not found - {file_path}")
        continue
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check if the expected columns exist
    if 'start' not in df.columns or 'end' not in df.columns:
        print(f"Error: Missing columns in file - {file_path}")
        continue
    
    # Update the 'start' and 'end' columns
    df['start'] = df['start'].apply(update_timestamps)
    df['end'] = df['end'].apply(update_timestamps)
    
    # Save the modified CSV file back to its original location
    df.to_csv(file_path, index=False)
    print(f"Processed and saved: {file_path}")

print("Task completed.")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.5. Inexistent Periods Raw Data
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Erasing those periors with the same start and end time .
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Due to some outage periords woul be shorter than the minimal time stept i.e. 15 minutes, some values may appear with the same value in is start and end time. 
<br>
Those ones are deported from the raw data files.
</div>

In [ ]:
# Define the reference data CSV file path
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Reference_Data.csv"

# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the column containing the file paths
column_to_process = 'Raw_Data_File_Path'

# Function to remove rows with same 'start' and 'end' values
def remove_same_start_end_rows(file_path):
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"Error: File not found - {file_path}")
        return
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check if the expected columns exist
    if 'start' not in df.columns or 'end' not in df.columns:
        print(f"Error: Missing columns in file - {file_path}")
        return
    
    # Remove rows where 'start' and 'end' values are the same
    df = df[df['start'] != df['end']]
    
    # Save the modified CSV file back to its original location
    df.to_csv(file_path, index=False)
    print(f"Processed and saved: {file_path}")

# Iterate over each file path in the specified column
for file_path in df_reference[column_to_process].dropna():
    remove_same_start_end_rows(file_path)

print("Task completed.")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.6. Time Step Raw Data
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Addapting the data into the minimal time stept.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The time series like the load data and the production of the units were got in a minimal time speo of 15 minutes e.g.
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
2023-01-01 00:15:00+00:00
<br>
2023-01-01 00:30:00+00:00
<br>
2023-01-01 00:45:00+00:00
<br>
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Since some of the outages from the sources are given in a diferent time stept e.g.
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
2023-01-01 00:37:00+00:00
<br>
2023-01-01 00:48:00+00:00
<br>
2023-01-01 00:52:00+00:00
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Those are going to be dadapted to the most closest value.
</div>

In [ ]:
# Function to round minutes to the nearest 15-minute interval
def round_to_nearest_15_minutes(timestamp_str):
    timestamp = datetime.fromisoformat(timestamp_str)
    minute = timestamp.minute
    
    if minute < 8:
        rounded_minutes = 0
    elif minute < 23:
        rounded_minutes = 15
    elif minute < 38:
        rounded_minutes = 30
    elif minute < 53:
        rounded_minutes = 45
    else:
        rounded_minutes = 0
        timestamp += timedelta(hours=1)  # Increment the hour

    rounded_timestamp = timestamp.replace(minute=rounded_minutes, second=0, microsecond=0)
    return rounded_timestamp.isoformat()

# Function to process a CSV file
def process_csv_file(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Round the 'start' and 'end' columns to the nearest 15 minutes
    df['start'] = df['start'].apply(round_to_nearest_15_minutes)
    df['end'] = df['end'].apply(round_to_nearest_15_minutes)
    
    # Save the modified CSV file back to its original location
    df.to_csv(file_path, index=False)
    print(f"Processed and saved: {file_path}")

# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the column to process
column_to_process = 'Raw_Data_File_Path'

# Iterate over each file path in the specified column
for file_path in df_reference[column_to_process].dropna():
    process_csv_file(file_path)

print("Task completed.")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.7. Duplicated Time Periods Raw Data 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Cleaning the outages that happened in the same time period.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Since each power plant can have more than one generator, the download data makes a sub-division accordingly, however, as the power plants data base are defined by a total power capacity, it is needed to set the outage data in the same way i.e. sum all the generators of the same power unit which outage time period is the same.
</div>

In [ ]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the column to process
column_to_process = 'Raw_Data_File_Path'

# Function to process each CSV file
def process_csv_file(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Group by the columns 'start', 'end', and 'production_resource_name'
    grouped = df.groupby(['start', 'end', 'production_resource_name'], as_index=False)

    # Sum the 'avail_qty' and 'nominal_power' columns
    df_summed = grouped.agg({
        'avail_qty': 'sum',
        'nominal_power': 'sum'
    })

    # Merge the summed columns back with the non-summed columns
    df_non_summed = grouped.head(1).drop(columns=['avail_qty', 'nominal_power'])
    df_result = pd.merge(df_non_summed, df_summed, on=['start', 'end', 'production_resource_name'])

    # Save the modified DataFrame back to a CSV file
    df_result.to_csv(file_path, index=False)

# Iterate over each file path in the specified column
for file_path in df_reference[column_to_process].dropna():
    process_csv_file(file_path)

print("Task completed.")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
5.8. Overlapped Outage Time Periods 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Separating those outages that has overlapped time periods.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Since each power plant can have more than one generator, the download data makes a sub-division accordingly, in consequence the outages periods of these generators are specified individually in the raw data.
<br>
This in some cases leads to the presence of overlapping time periods that has to be separated.
</div>

In [ ]:
# Load the reference CSV file
reference_df = pd.read_csv(reference_data_file_path)

# Function to handle overlaps and create new rows
def handle_overlaps(df):
    # Convert start and end columns to datetime
    df['start'] = pd.to_datetime(df['start'])
    df['end'] = pd.to_datetime(df['end'])

    new_rows = []

    # Iterate over each group of production_resource_name
    for name, group in df.groupby('production_resource_name'):
        group = group.sort_values(by=['start'])
        while not group.empty:
            row = group.iloc[0]
            group = group.iloc[1:]
            
            overlapping_rows = group[(group['start'] < row['end']) & (group['end'] > row['start'])]
            if not overlapping_rows.empty:
                overlap_start = max(row['start'], overlapping_rows['start'].min())
                overlap_end = min(row['end'], overlapping_rows['end'].max())

                overlap_avail_qty = row['avail_qty'] + overlapping_rows['avail_qty'].sum()
                overlap_nominal_power = row['nominal_power'] + overlapping_rows['nominal_power'].sum()

                new_rows.append({
                    'avail_qty': overlap_avail_qty,
                    'nominal_power': overlap_nominal_power,
                    'production_resource_name': name,
                    'start': overlap_start,
                    'end': overlap_end
                })

                if row['start'] < overlap_start:
                    new_rows.append({
                        'avail_qty': row['avail_qty'],
                        'nominal_power': row['nominal_power'],
                        'production_resource_name': name,
                        'start': row['start'],
                        'end': overlap_start
                    })

                if row['end'] > overlap_end:
                    new_rows.append({
                        'avail_qty': row['avail_qty'],
                        'nominal_power': row['nominal_power'],
                        'production_resource_name': name,
                        'start': overlap_end,
                        'end': row['end']
                    })

                group = group[~((group['start'] < row['end']) & (group['end'] > row['start']))]

            else:
                new_rows.append(row.to_dict())

    # Create a new DataFrame from the new_rows
    new_df = pd.DataFrame(new_rows)

    return new_df

# Iterate over each file path in the reference CSV file
for file_path in reference_df['Raw_Data_File_Path']:
    try:
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Process the dataframe to handle overlaps
        df_result = handle_overlaps(df)
        
        # Save the result back to the CSV file
        df_result.to_csv(file_path, index=False)

        print(f"Processed and saved file: {file_path}")
    except Exception as e:
        print(f"An error occurred while processing file {file_path}: {e}")

print("Task completed.")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
6. Outage Factors Calculation 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Getting the Outage Factors for each time period.
</div>
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
6.1. Raw Data Sources File Verification
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Verifying the column names of the raw data
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The web source used does not allways have the data from all countries e.g. the Cypres case. so this files are empty.
    <br>
Hence the same column names are going to be armonized to all the raw data files even those that are empty.
    <br>
This is needed to avoid errors in the next cells 
</div>

In [ ]:
# Load the reference data CSV file
reference_df = pd.read_csv(reference_data_file_path)

# Function to check if a CSV file is empty or has no headers
def is_csv_empty_or_no_headers(file_path):
    if not os.path.exists(file_path):
        print(f"File does not exist: {file_path}")
        return True
    
    with open(file_path, 'r') as f:
        first_line = f.readline().strip()
        if first_line == '':
            return True
        else:
            return False

# Function to copy headers to an empty CSV file
def copy_headers(source_file, target_file):
    headers = pd.read_csv(source_file, nrows=0)
    headers.to_csv(target_file, index=False)
    print(f"Copied headers from {source_file} to {target_file}")

# Clean the dataframe to remove NaN values in the 'Raw_Data_File_Path' column
reference_df = reference_df.dropna(subset=['Raw_Data_File_Path'])

# Iterate through each row in the reference dataframe to find a non-empty CSV file with headers
non_empty_csv_with_headers = None
for file_path in reference_df['Raw_Data_File_Path']:
    file_path = str(file_path)
    if os.path.exists(file_path) and not is_csv_empty_or_no_headers(file_path):
        non_empty_csv_with_headers = file_path
        break

# Check if we have found a non-empty CSV file with headers
if non_empty_csv_with_headers:
    print(f"Found non-empty CSV file with headers: {non_empty_csv_with_headers}")
    
    # Iterate through each row in the reference dataframe
    for file_path in reference_df['Raw_Data_File_Path']:
        file_path = str(file_path)
        if os.path.exists(file_path) and is_csv_empty_or_no_headers(file_path):
            copy_headers(non_empty_csv_with_headers, file_path)
else:
    print("No non-empty CSV file with headers found to copy from.")

print("Header copy operation completed.")


<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
6.2. Power Plants Data 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Getting the power plants data list.
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The total power capacity of each unit is required to calculate the Outage Factos.
<br>
The corresponding paths to each of the files where these informations is located is extrated to list called 'power_plants_data_file_path'
</div>

In [ ]:
power_plants_data_file_path = []  # Empty list to store power plant data file paths

# Open the reference data file
with open(reference_data_file_path, 'r') as reference_data_file:
  reader = csv.DictReader(reference_data_file)

  # Loop through each row in the reference data file
  for row in reader:
    dispa_set_code = row['Dispa-SET_Code']

    # Construct the path to the power plant data file
    power_plant_file_path = f"{power_plants_data_folder_path}{dispa_set_code}/{data_year}.csv"

    # Append the path to the list
    power_plants_data_file_path.append(power_plant_file_path)

print(f"Power plant data file paths extracted: {power_plants_data_file_path}")

power_plants_data_file_path

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Updating the reference file.
<div/>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The reference_data_file_path define a csv file that contains all the paths of the csv files used to get the final formatted outages factors data.
The same was created to manage the data in orderly way so the power_plants_data_file_path list is goint to be copiet to the reference file.
</div>

In [ ]:
# Load the reference data
reference_df = pd.read_csv(reference_data_file_path)

# Ensure the list and reference DataFrame have the same length
if len(power_plants_data_file_path) != len(reference_df):
    raise ValueError("The length of power_plants_data_file_path and the reference data must be the same")

# Add the new column to the DataFrame
reference_df['Power_Plants_Data'] = power_plants_data_file_path

# Save the updated DataFrame back to the CSV file
reference_df.to_csv(reference_data_file_path, index=False)

print("Updated the reference data file with the new column 'Power_Plants_Data'.")

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [ ]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"universal_standard_time:                                   {universal_standard_time}")
print (f"western_european_time:                                     {western_european_time}")
print (f"central_european_time:                                     {central_european_time}")
print (f"eastern_european_time:                                     {eastern_european_time}")
print (f"reference_data_folder_path:                                {reference_data_folder_path}")
print (f"reference_data_file_name:                                  {reference_data_file_name}")
print (f"reference_data_file_path:                                  {reference_data_file_path}")
print (f"outage_factors_folder_path:                                {outage_factors_folder_path}")
print(f"outage_factors_raw_data_folder_path:                        {outage_factors_raw_data_folder_path}")
print(f"power_plants_data_folder_path:                              {power_plants_data_folder_path}")
print(f"power_plants_data_file_path:                                {power_plants_data_file_path}")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
6.3. Total Power Plant Capacity 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Copying the power plant capacity values.
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The corresponding total/nominal capacity of the units are copied from the already formatted power plants data file in the respective analized year.
<br>
These values later are used to calculate the outage factor over each time period of unavailability of the generation units.
</div>

In [ ]:
# Load the reference data CSV file
reference_df = pd.read_csv(reference_data_file_path)

# Check if the number of entries in the reference file matches the length of the power plants data file paths list
if len(reference_df) != len(power_plants_data_file_path):
    raise ValueError("The number of entries in the reference file does not match the number of power plants data file paths")

# Iterate through each pair of Raw_Data_File_Path and power plants data file path
for index, row in reference_df.iterrows():
    file_path_1 = row['Raw_Data_File_Path']
    file_path_2 = power_plants_data_file_path[index]

    # Load the CSV files into dataframes
    df1 = pd.read_csv(file_path_1)
    df2 = pd.read_csv(file_path_2)

    # Create a new column 'Total_Capacity' in df1 and initialize it with NaN
    df1['Total_Capacity'] = pd.NA

    # Iterate over each row in df2
    for _, row in df2.iterrows():
        unit_name = row['Unit']
        power_capacity = row['PowerCapacity']

        # Find matching rows in df1
        matching_rows = df1[df1['production_resource_name'] == unit_name]

        if not matching_rows.empty:
            # Update the 'Total_Capacity' in the matching rows
            df1.loc[matching_rows.index, 'Total_Capacity'] = power_capacity

    # Save the updated dataframe back to the original CSV file
    df1.to_csv(file_path_1, index=False)

    print(f"Update completed for {file_path_1}.")

print("All updates completed.")

<div style="text-align: justify; margin-left: 2.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
6.4. Raw Data Files Outage Factors Calculation 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Getting the corresponding Outage Factors in a new column inside the raw data files.
</div>

In [ ]:
# Load the reference data CSV file
reference_df = pd.read_csv(reference_data_file_path)

# Iterate through each row in the reference dataframe
for file_path in reference_df['Raw_Data_File_Path']:
    # Check if the file exists
    if os.path.exists(file_path):
        # Load the target CSV file
        df = pd.read_csv(file_path)

        # Ensure the required columns exist
        if 'avail_qty' in df.columns and 'nominal_power' in df.columns and 'Total_Capacity' in df.columns:
            # Calculate the highest value between 'nominal_power' and 'Total_Capacity'
            max_power_capacity = df[['nominal_power', 'Total_Capacity']].max(axis=1)

            # Calculate the Outage Factor
            df['Outage_Factor'] = df['avail_qty'] / max_power_capacity

            # Save the updated dataframe back to the CSV file
            df.to_csv(file_path, index=False)
            print(f"Processed file: {file_path}")
        else:
            print(f"Missing required columns in file: {file_path}")
    else:
        print(f"File does not exist: {file_path}")

print("Processing completed.")

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
6. Outage Factors Files
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Getting the final formated files.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
6.1. Clean Data Files
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Creating the files where the clean outage data is going to be storaged.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The Outage Factors data has to be added using a determined time step as well, the same is going to be extracted from the Availability Factors already formated data from the corresponding year.
</div>

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv(reference_data_file_path)

# Add new columns to the DataFrame with initial values (e.g., NaN or some default value)
df['1h'] = pd.NA  # or use df['1h'] = 0 if you want to initialize with zeros
df['30min'] = pd.NA  # or use df['30min'] = 0
df['15min'] = pd.NA  # or use df['15min'] = 0

# Save the updated DataFrame back to a CSV file
df.to_csv(reference_data_file_path, index=False)

# If you want to save it to a new file instead of overwriting the original one, you can use:
# df.to_csv("/home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Updated_Reference_Data.csv", index=False)

In [ ]:
# Read the reference CSV file
df = pd.read_csv(reference_data_file_path)

# Get the list of Dispa-SET_Code values
dispa_set_codes = df['Dispa-SET_Code'].unique()

# Copy the folders and their contents
for code in dispa_set_codes:
    src_folder = os.path.join(availability_factors_data_folder_path, code)
    dest_folder = os.path.join(outage_factors_folder_path, code)
    
    # Check if the source folder exists
    if os.path.exists(src_folder):
        # Copy the folder to the destination
        shutil.copytree(src_folder, dest_folder, dirs_exist_ok=True)
        print(f"Copied folder: {code}")
        
        # Remove unnecessary files in the copied folder
        for filename in os.listdir(dest_folder):
            file_path = os.path.join(dest_folder, filename)
            if filename != f"{data_year}.csv":
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    print(f"Deleted file: {file_path}")
    else:
        print(f"Source folder does not exist: {src_folder}")

print("Task completed.")

In [ ]:
# Read the reference CSV file
df = pd.read_csv(reference_data_file_path)

# Get the list of Dispa-SET_Code values
dispa_set_codes = df['Dispa-SET_Code'].unique()

# Columns to check
columns = ['1h', '30min', '15min']

# Process each Dispa-SET_Code folder
for code in dispa_set_codes:
    folder_path = os.path.join(outage_factors_folder_path, code)
    
    if os.path.exists(folder_path):
        for col in columns:
            subfolder_path = os.path.join(folder_path, col)
            if os.path.exists(subfolder_path):
                for filename in os.listdir(subfolder_path):
                    file_path = os.path.join(subfolder_path, filename)
                    if filename != f"{data_year}.csv":
                        if os.path.isfile(file_path):
                            os.remove(file_path)
                            print(f"Deleted file: {file_path}")
                    else:
                        # Update the reference DataFrame with the path of the remaining CSV file
                        df.loc[df['Dispa-SET_Code'] == code, col] = file_path
    else:
        print(f"Folder does not exist: {folder_path}")

# Save the updated DataFrame back to the reference CSV file
df.to_csv(reference_data_file_path, index=False)

print("Task completed.")

In [ ]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the columns to process
columns_to_process = ['1h', '30min', '15min']

# Iterate over the specified columns
for column in columns_to_process:
    # Iterate over each file path in the column
    for file_path in df_reference[column].dropna():
        # Read the CSV file
        df = pd.read_csv(file_path)
        # Keep only the first column
        df = df.iloc[:, :1]
        # Save the modified CSV file
        df.to_csv(file_path, index=False)
        print(f"Processed: {file_path}")

print("Task completed.")

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
6.2. Clean Data Files Headers
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Copying the power units to the Outage Factors clean data files.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The Outage Factors data files have to have all the power units name as headers to identify the corresponding Outage Factor belonging.
<br>
These names are copied from the already formated Power Plants files from the dispa-SET data base directory in the analized year.
</div>

In [ ]:
# Define the path to the reference data file
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/OutageFactors/Reference_Data.csv"

# Load the reference data
reference_df = pd.read_csv(reference_data_file_path)

# Iterate over each row in the reference data
for index, row in reference_df.iterrows():
    power_plants_data_path = row['Power_Plants_Data']
    
    # Check if the power plants data file exists
    if isinstance(power_plants_data_path, str) and os.path.exists(power_plants_data_path):
        # Load the power plants data file
        power_plants_df = pd.read_csv(power_plants_data_path)
        
        # Extract the 'Unit' column values
        unit_values = power_plants_df['Unit'].tolist()
        
        # Iterate over the columns '1h', '30min', '15min'
        for time_column in ['1h', '30min', '15min']:
            dest_file_path = row[time_column]
            
            # Check if the destination file path is valid and exists
            if isinstance(dest_file_path, str) and os.path.exists(dest_file_path):
                # Read the destination file
                dest_df = pd.read_csv(dest_file_path)
                
                # Create a new DataFrame with unit values as header
                unit_df = pd.DataFrame(columns=unit_values)
                
                # Append the existing rows from the destination file to the new DataFrame
                # and ignore the index to keep the original indexing
                updated_df = pd.concat([dest_df, unit_df], axis=1)
                
                # Save the updated DataFrame back to the destination file
                updated_df.to_csv(dest_file_path, index=False)
                
                print(f"Updated file: {dest_file_path} with new headers from {power_plants_data_path}")
            else:
                print(f"Invalid or non-existent file path: {dest_file_path}, skipping.")
    else:
        print(f"Invalid or non-existent file path: {power_plants_data_path}, skipping.")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Giving the corresponding first column header.
</div>

In [ ]:
def modify_csv_file(file_path):
    """
    Modifies a CSV file by duplicating the last row and shifting existing rows down.

    Args:
        file_path: The path to the CSV file.
    """

    df = pd.read_csv(file_path)

    # Duplicate the last row and append it to the DataFrame
    df = pd.concat([df, df.iloc[-1:]], ignore_index=True)

    # Shift existing rows down in the first column
    df.iloc[1:-1, 0] = df.iloc[:-2, 0].copy()

    # Copy the header to the first row
    df.iloc[0] = df.columns

    # Rename the first column
    df.columns = ['Unnamed: 0'] + df.columns[1:].tolist()

    # Clear values in all rows except for the first column
    df.iloc[:, 1:] = ''

    df.to_csv(file_path, index=False)
    print(f"Processed file: {file_path}")

def process_files(reference_data_file_path):
    """
    Processes CSV files based on information in the specified file.

    Args:
        reference_data_file_path: The path to the CSV file containing file paths.
    """

    df = pd.read_csv(reference_data_file_path)

    for index, row in df.iterrows():
        for column in ['1h', '30min', '15min']:
            file_path = row[column]
            if pd.notnull(file_path):
                modify_csv_file(file_path)

process_files(reference_data_file_path)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
6.3. Clean Outage Factors Data Files
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Copying all the already calculated outages factors to the clean files.
</div>

In [ ]:
reference_data = pd.read_csv(reference_data_file_path)

# Iterate over each row in the reference data
for _, row in reference_data.iterrows():
    raw_data_file_paths = [row['Raw_Data_File_Path'], row['1h'], row['30min'], row['15min']]
    
    # Skip if any field is empty
    if any(pd.isnull(file_path) for file_path in raw_data_file_paths):
        continue
    
    # Load file paths
    file_path_1 = raw_data_file_paths[0]
    file_paths_2 = raw_data_file_paths[1:]

    # Load CSV files
    df1 = pd.read_csv(file_path_1)
    
    # Convert 'start' and 'end' columns to datetime
    df1['start'] = pd.to_datetime(df1['start'])
    df1['end'] = pd.to_datetime(df1['end'])

    # Iterate over file paths for file_path_2
    for file_path_2 in file_paths_2:
        df2 = pd.read_csv(file_path_2)
        df2['Unnamed: 0'] = pd.to_datetime(df2['Unnamed: 0'])

        # Iterate through each row in df1
        for _, row in df1.iterrows():
            start_time = row['start']
            end_time = row['end']
            outage_factor = row['Outage_Factor']
            production_resource_name = row['production_resource_name']

            # Filter df2 to rows within the time period
            filtered_rows = df2[(df2['Unnamed: 0'] >= start_time) & (df2['Unnamed: 0'] <= end_time)]

            # Iterate through filtered rows
            for _, filtered_row in filtered_rows.iterrows():
                # Paste the 'Outage_Factor' value into corresponding rows of df2
                df2.at[filtered_row.name, production_resource_name] = outage_factor

        # Save modified df2 back to CSV
        df2.to_csv(file_path_2, index=False)

print("Completed updating the CSV files.")

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Filling with a Outage Factor of '0' to all those power unit that do not have any unavailability in some time period.4
<br>
This is done to all the Dispa-SET countries.
</div>

In [ ]:
reference_data = pd.read_csv(reference_data_file_path)

# Iterate over each column in the reference data ('1h', '30min', '15min')
for column in ['1h', '30min', '15min']:
    # Iterate over each unique file path in the current column
    for file_path in reference_data[column].dropna().unique():
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the CSV file has columns other than 'Unnamed: 0'
        if len(df.columns) > 1:
            # Fill empty fields of all columns except the first one with value '1'
            df.iloc[:, 1:] = df.iloc[:, 1:].fillna(0)
            
            # Save the modified CSV file
            df.to_csv(file_path, index=False)
            print(f"Filled empty fields in {file_path}")
        else:
            print(f"Skipping {file_path} as it only has the 'Unnamed: 0' column")

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
7. Copying Outage Factors Formatted Data Files
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Moving the already clean and formatted data to the main Dispa-SET data base directory.
     <br>
    <div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The data base directory has to be identified.
</div>

In [ ]:
additional_path_4 = "/Database/OutageFactors/"

# Construct the power_plants_raw_data_folder_path variable
outages_factor_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_4

In [ ]:
def process_reference_data(reference_data_file_path, outages_factor_data_base_folder_path):
    """
    Processes the reference data and copies CSV files to corresponding subfolders, overwriting existing files.

    Args:
        reference_data_file_path: The path to the reference data CSV file.
        outages_factor_data_base_folder_path: The path to the base folder for outage factors.
    """

    reference_data = pd.read_csv(reference_data_file_path)

    for index, row in reference_data.iterrows():
        dispa_set_code = row['Dispa-SET_Code']
        source_folder_path = os.path.join(outages_factor_data_base_folder_path, dispa_set_code)

        if os.path.exists(source_folder_path) and os.path.isdir(source_folder_path):
            # Ensure subfolders exist
            for subfolder in ['1h', '30min', '15min']:
                subfolder_path = os.path.join(source_folder_path, subfolder)
                if not os.path.exists(subfolder_path):
                    os.makedirs(subfolder_path)

            # Copy CSV files, overwriting existing ones
            for subfolder in ['1h', '30min', '15min']:
                if not pd.isnull(row[subfolder]):
                    source_file_path = row[subfolder]
                    destination_file_path = os.path.join(source_folder_path, subfolder, os.path.basename(source_file_path))
                    shutil.copy2(source_file_path, destination_file_path)
                    print(f"Copied {os.path.basename(source_file_path)} to {destination_file_path}")
        else:
            print(f"Skipping empty or non-existent folder: {source_folder_path}")

process_reference_data(reference_data_file_path, outages_factor_data_base_folder_path)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
7. Outages Factors Folder Back Up
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Once all the formating process was done the Outages Factors Folder is restored to its defoult state.
</div>

In [ ]:
if os.path.exists(outage_factors_folder_path):
    shutil.rmtree(outage_factors_folder_path)  # Remove the current directory
shutil.copytree(backup_folder_path, outage_factors_folder_path)

print(f"Directory restored to original state from {backup_folder_path}")

In [ ]:
shutil.rmtree(backup_folder_path)
print(f"Backup folder {backup_folder_path} deleted successfully.")